In [1]:
import pandas as pd
import seaborn as sns
import numpy as np

pd.set_option('display.max_columns', None)

In [2]:
dados = pd.read_json('../data/raw/vendas_zap_rj_fev_2022_corte_original.json', lines=True)
dados.head(1)

,listing,account,medias,accountLink,link
0,"{'displayAddressType': 'ALL', 'amenities': [],...","{'id': '93af6988-7e67-d3c3-c81f-183e982f9f33',...","[{'id': 'a9cf07f6cb7580abb4fb9087b6435404', 'u...","{'data': {}, 'name': 'MARCIO DE SOUZA QUEIROZ'...","{'data': {'zone': 'Zona Oeste', 'neighborhood'..."


In [3]:
dados2 = pd.json_normalize(dados['listing']).explode('pricingInfos').reset_index(drop=True)  
dados2.head(1)

,displayAddressType,amenities,usableAreas,constructionStatus,listingType,description,title,createdAt,floors,unitTypes,nonActivationReason,providerId,propertyType,unitSubTypes,unitsOnTheFloor,legacyId,id,portal,unitFloor,parkingSpaces,updatedAt,suites,publicationType,externalId,bathrooms,usageTypes,totalAreas,advertiserId,whatsappNumber,bedrooms,acceptExchange,pricingInfos,showPrice,resale,buildings,capacityLimit,status,address.country,address.zipCode,address.geoJson,address.city,address.streetNumber,address.level,address.precision,address.confidence,address.stateAcronym,address.source,address.point.lon,address.point.source,address.point.lat,address.ibgeCityId,address.zone,address.street,address.locationId,address.district,address.name,address.state,address.neighborhood,address.poisList,address.complement,address.pois,address.valuableZones,advertiserContact.phones,advertiserContact.chat
0,ALL,[],[150],ConstructionStatus_NONE,USED,Vendo lote residencial em Santa Cruz. Próximo ...,Vendo lote residencial em Santa Cruz,2021-09-02T12:26:50.814Z,[],[RESIDENTIAL_ALLOTMENT_LAND],NonActivationReason_NONE,,UNIT,[],0,,2533215210,GRUPOZAP,0,[0],2022-02-05T11:06:29.117Z,[0],PREMIUM,9BB24B,[0],[RESIDENTIAL],[150],93af6988-7e67-d3c3-c81f-183e982f9f33,21996030404,[0],False,"{'yearlyIptu': '0', 'price': '3500', 'business...",True,False,0.0,[],ACTIVE,BR,23560352,,Rio de Janeiro,2000,STREET,RANGE_INTERPOLATED,VALID_STREET,RJ,CORREIOS,-43.704318,GOOGLE,-22.906731,,Zona Oeste,Avenida João XXIII,BR>Rio de Janeiro>NULL>Rio de Janeiro>Zona Oes...,,,Rio de Janeiro,Santa Cruz,"[CS:Mercado Tropical, CS:Mercado Novo Horizont...",,[],[],[21996030404],NaN


In [4]:
dados_tratados = pd.concat([dados2, pd.json_normalize(dados2['pricingInfos'])], axis=1).drop(['pricingInfos'], axis=1)  
dados_tratados = dados_tratados[dados_tratados['businessType'] == 'SALE']  # filtra apenas vendas
dados_tratados = dados_tratados[dados_tratados['address.state'] != 'Rio Grande do Sul']  # Há um registro referente ao estado RS  
dados_tratados.reset_index(drop=True, inplace=True)

In [5]:
# Escolhendo as features que serão trabalhadas:

dados_tratados = dados_tratados[['unitTypes',
                                'usageTypes',
                                'usableAreas',
                                'totalAreas',
                                'bedrooms',
                                'bathrooms',
                                'parkingSpaces',
                                'suites',
                                'address.point.lon',
                                'address.point.lat',
                                'address.zone',
                                'address.neighborhood',
                                'yearlyIptu',
                                'monthlyCondoFee',
                                'price']].rename(columns={'address.city': 'city',
                                                          'address.point.lon': 'longitude',
                                                          'address.point.lat': 'latitude',
                                                          'address.zone': 'zone',
                                                          'address.neighborhood': 'neighborhood'})
                                                          
                                                          
                                                          

In [6]:
def explode_columns(dataframe, list_columns: list):
    df_copy = dataframe.copy()
    for column in list_columns:
        df_copy = df_copy.explode(column)
    return df_copy

In [7]:
columns = ['unitTypes', 'usageTypes', 'usableAreas', 'totalAreas', 'bedrooms', 'bathrooms', 'parkingSpaces', 'suites']
dataframe_final = explode_columns(dados_tratados, columns)
dataframe_final

,unitTypes,usageTypes,usableAreas,totalAreas,bedrooms,bathrooms,parkingSpaces,suites,longitude,latitude,zone,neighborhood,yearlyIptu,monthlyCondoFee,price
0,RESIDENTIAL_ALLOTMENT_LAND,RESIDENTIAL,150,150,0,0,0,0,-43.704318,-22.906731,Zona Oeste,Santa Cruz,0,0,3500
1,APARTMENT,RESIDENTIAL,42,42,2,1,0,0,-43.203115,-22.896929,Zona Central,Santo Cristo,100,400,18500
2,APARTMENT,RESIDENTIAL,53,53,2,3,2,2,-43.173045,-22.924791,Zona Sul,Flamengo,NaN,NaN,18000
3,PARKING_SPACE,COMMERCIAL,12,12,0,0,1,NaN,-43.178923,-22.965244,Zona Sul,Copacabana,116,300,20000
4,APARTMENT,RESIDENTIAL,42,NaN,1,1,NaN,0,-43.222759,-22.888793,Zona Norte,São Cristóvão,110,300,15000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80430,HOME,RESIDENTIAL,1024,1024,5,10,4,1,-43.388906,-23.003960,Zona Oeste,Barra da Tijuca,2900,2350,10000000
80431,HOME,RESIDENTIAL,750,1250,5,7,6,5,-43.395032,-23.000902,Zona Oeste,Barra da Tijuca,15000,3000,10000000
80432,APARTMENT,RESIDENTIAL,328,328,4,5,3,4,-43.189682,-22.987018,Zona Sul,Copacabana,NaN,NaN,10000000
80433,HOME,RESIDENTIAL,1024,1024,5,10,4,5,-43.351836,-23.005037,Zona Oeste,Barra da Tijuca,35000,2400,10000000


[Nullable interger](https://pandas.pydata.org/pandas-docs/stable/user_guide/integer_na.html#nullable-integer-data-type)

In [8]:
dataframe_final = dataframe_final.astype({'usableAreas': 'Int32',
                                          'totalAreas': 'Int32',
                                          'bedrooms': 'Int32',
                                          'bathrooms': 'Int32',
                                          'parkingSpaces': 'Int32',
                                          'suites': 'Int32',
                                          'yearlyIptu': 'Int32',
                                          'monthlyCondoFee': 'Int32',
                                          'price': 'Int32'})

In [9]:
dataframe_final.dtypes

unitTypes           object
usageTypes          object
usableAreas          Int32
totalAreas           Int32
bedrooms             Int32
bathrooms            Int32
parkingSpaces        Int32
suites               Int32
longitude          float64
latitude           float64
zone                object
neighborhood        object
yearlyIptu           Int32
monthlyCondoFee      Int32
price                Int32
dtype: object

In [10]:
dataframe_final.describe().round(5)

,usableAreas,totalAreas,bedrooms,bathrooms,parkingSpaces,suites,longitude,latitude,yearlyIptu,monthlyCondoFee,price
count,8.042700e+04,7.276000e+04,79926.00000,80273.00000,77203.00000,74060.00000,80288.00000,80288.00000,6.985300e+04,7.262300e+04,8.043600e+04
mean,1.926906e+03,3.447573e+03,2.64266,2.46169,1.53923,1.26129,-43.31340,-22.95616,6.133865e+03,3.512424e+03,1.362901e+06
std,3.039080e+05,3.563501e+05,1.29990,1.61415,1.85017,1.28016,0.13297,0.10091,3.582950e+05,7.933194e+04,1.554858e+06
min,0.000000e+00,0.000000e+00,0.00000,0.00000,0.00000,0.00000,-43.70744,-43.51913,0.000000e+00,0.000000e+00,3.500000e+03
25%,6.700000e+01,6.900000e+01,2.00000,1.00000,1.00000,0.00000,-43.38675,-22.99204,6.000000e+01,4.000000e+02,4.200000e+05
50%,9.600000e+01,1.000000e+02,3.00000,2.00000,1.00000,1.00000,-43.31973,-22.96520,4.000000e+02,8.020000e+02,7.800000e+05
75%,1.690000e+02,1.800000e+02,3.00000,3.00000,2.00000,2.00000,-43.20932,-22.92499,1.580000e+03,1.446000e+03,1.650000e+06
max,7.881638e+07,7.881638e+07,50.00000,50.00000,180.00000,50.00000,-22.92817,-8.14000,7.512200e+07,9.900000e+06,1.000000e+07


In [11]:
# Salvando o conjunto de dados tratados em um arquivo csv:

dataframe_final.to_csv("../data/processed/dados_tratados.csv", index=False, sep=';')